In [ ]:
%pylab inline
import numpy as np
from random import random
from scipy.interpolate import interp1d
from astroML.correlation import two_point
import matplotlib.pyplot as plt
#We use astroML function two_point. 

In this notebook we show a simple way to use astroML libray to compute the two_point correlation function
on a simulated data set. The simulation is a cosmological box of $L=512{\rm  Mpc} h^{-1}$ (we'll omit the h everywhere), and $128^3$ particles. The cosmological model used was a LCDM. 

In [ ]:
#Read the true power spectrum used to generate the simulation. 
Pk=np.loadtxt('./input_spectrum.dat')
pk_=interp1d(Pk[:,0],Pk[:,1])
kh=10**np.linspace(-4,2,5000)
def xi_true():
    pk_h=pk_(kh)
    # Compute the correlation  function from the true  P(k) used in the simulation. Fourier Transform. 
    kstart =0
    kcut = 10
    kcut_ind = np.argmin(abs(kh - kcut))
#print 'kcut_ind=', kcut_ind
#print 'kcut=', kh[kcut_ind]

    r_start = 1
    r_end = 150
    r_stepsize = 1
    r_list = np.arange(r_start, r_end, r_stepsize)

    xi = np.zeros(len(r_list))
    factor = np.power(kh[kstart:kcut_ind],2) * pk_h[kstart:kcut_ind] / (kh[kstart:kcut_ind]*2*np.pi**2)
    for i in range(0, len(r_list)):
        IntegrandXi0 = factor * np.sin(kh[kstart:kcut_ind]*r_list[i]) / r_list[i]
        xi[i] = np.trapz(IntegrandXi0,kh[kstart:kcut_ind])
    return r_list, xi

In [ ]:
#Plot the mass power spectrum. Just to see how it looks like... 
plt.plot(kh,pk_(kh))
plt.yscale('log')
plt.xscale('log')
plt.ylim(1e-4,1e5)
plt.xlim(2e-5,2e2)
plt.xlabel('k')
plt.ylabel('P(k)')

In [ ]:
#Read the data, note that here I'm using my original files,
#not those I shared with you. Which are just N random points from the full box. 
#This data is separated in a 10 slices of ~50Mpc. 


path='./' #path to where the files are located. 
data=np.loadtxt(path+'example_128_z0p000.0')
print(len(data))
for i in range(1,10):
    data_=np.loadtxt(path+'example_128_z0p000.'+np.str(i))
    data=np.vstack((data,data_))

In [ ]:
#Plot all points projected in the xy plane.  
fig=plt.figure(figsize=(10,10))
plt.scatter(x,y,s=0.1,color='blue',alpha=0.05)
plt.xlim(0,512)
plt.ylim(0,512)
plt.xlabel('x[Mpc]')
plt.ylabel('y [Mpc]')

In [ ]:
#Since I read the full data set it is about 2M particcles. 
#To compute the CF for this could take really long so I'll use only a random subset of the particles.
# This is what I did also to produce the smaller files I shared. 
#Calculation with 250K takes a while, so I'd suggest to start with less. 
w=np.random.choice(len(data),50000)
#Notice this data has both positions and velocities, we only use positions. 
pos=[[x,y,z] for [x,y,z,vx,vy,vz] in data[w]]
pos=np.asarray(pos)
#fname="prueba_10k"
#np.save(fname,pos)

In [ ]:
#Now, compute the correlation function for the 1<r<150 with the Standard estimator.
bins=np.linspace(1.,130,40)
corr=two_point(pos,bins,method='standard')

In [ ]:
#Now, compute the correlation function for the 1<r<150 with the Landy-Szalay estimator.
corr2=two_point(pos,bins,method='landy-szalay')

In [ ]:
#Now, compute the correlation function for the 1<r<150 with the Landy-Szalay method.
r=0.5*(bins[1:] + bins[:-1])

r2corr=np.power(r,2)*(corr)
r2corr2=np.power(r,2)*(corr2)


plot(r,r2corr,label='Standard')
plot(r,r2corr2,color='r',label='Landy')

xlim(0,200)
ylim(-20,50)
plt.legend()


In [ ]:
r_,xi=xi_true()

plt.plot(r,r2corr,label='Standard')
plt.plot(r,r2corr2,color='r',label='Landy')
plt.plot(r_, np.power(r_,2)*xi ,label='linear')
#legend()
plt.xlim(0,200)
plt.ylim(-20,50)
plt.xlabel(r'$r[Mpc]$')
plt.ylabel(r'$r^2\xi(r)$')
plt.legend()

